In [ ]:
import sys
# Please download the Opentrons API and save into the same workspace
from opentrons import protocol_api, execute, simulate
from opentrons.types import Location, Point
import pandas as pd
import numpy as np
import urllib.request
import time
import ast  # safer than eval
#imports take roughly 1 minute

# Before running the program, please setup a server connection for a shared .csv folder. 

After connecting the OT-2 to the same network/LAN,
Run this in the terminal of the central computer: 

```
cd: "your protocol folder path"
python -m http.server 8000
```

This will open a sever to allow opentron to acess the file from: 

csv_path = "http://your pc IP HERE:8000/volumes.csv"

In [7]:
# example volume for red, yellow, blue, black pattern
path = "http://192.168.0.117:8000/Volumes96.csv"
df = pd.read_csv(path)
df # Display volumes to verify

# Define functions

In [8]:
# metadata
metadata = {
    "protocolName": "IMOD Protocol",
    "author": "Clara Tamura, Austin Martin",
    "description": "Mara OT-2 Protocol for IMOD project",
    "apiLevel": "2.19",
}

class OpentronMara:
    def __init__(self, protocol):
        self.protocol = protocol
        self.protocol._commands.clear()

        # Load tipracks
        self.tiprack_300 = protocol.load_labware('opentrons_96_tiprack_300ul', '2')

        # Load Pipettes
        self.pip300 = protocol.load_instrument('p300_single_gen2', mount='right', tip_racks=[self.tiprack_300])

        # Track used tips manually
        self.tip_log = {
            self.pip300: {'tips': self.tiprack_300.wells(), 'next': 0},
        }

        # Set pipette buffers
        self.pip300_buffer = 20

        self.pip300.flow_rate.aspirate = 50
        self.pip300.flow_rate.dispense = 50
        self.pip300.flow_rate.blow_out = 100

        # Load plates
        self.plate_96 = protocol.load_labware('opentrons_96_wellplate_200ul_pcr_full_skirt', '1')
        self.plate_96.set_offset(x=0, y=0, z=55)  # Z lowered by 1 mm
    
        self.plate_6 = protocol.load_labware('corning_6_wellplate_16.8ml_flat','3')
        self.plate_6.set_offset(x=0, y=0, z=10)
        self.source_wells = ['A1', 'A2', 'B1', 'B2']

        # Home the pipettes
        self.pip300.home()

        print("Initialization complete.")

    def pick_up_tip(self, pipette):
        """Pick up the next available tip for the given pipette, with rack limit check."""
        next_tip_index = self.tip_log[pipette]['next']
        tips = self.tip_log[pipette]['tips']
        
        if next_tip_index >= len(tips):
            raise RuntimeError(f"❌ No more tips available for {pipette.name}. Tip rack is empty!")

        tip = tips[next_tip_index]
        pipette.pick_up_tip(tip)
        self.tip_log[pipette]['next'] += 1

    def reset_tips(self, pipette=None):
        if pipette:
            self.tip_log[pipette]['next'] = 0
        else:
            for pip in self.tip_log:
                self.tip_log[pip]['next'] = 0
        
    def distribute_colors(self, df_combined: pd.DataFrame, columns):
        self.columns = columns
        volumes_df = df_combined[self.columns].round(1)
        num_samples, num_solutions = volumes_df.shape
        well_names = df_combined['well']
        print(well_names)
        destination = [self.plate_96.wells_by_name()[well] for well in well_names]

        
        for i in range(num_solutions):
            self.pip300.distribute(
                volumes_df[self.columns[i]].to_list(),
                self.plate_6.wells(self.source_wells[i])[0],
                destination,
                disposal_vol=0,
                blow_out=True,
                blowout_location='source well',
                new_tip='once',
                trash=False
            )
        self.reset_tips(self.pip300) # Reset the tip log for the pipette after use
        #self.print_cmd()
        
        
    def print_cmd(self):
        print("Printing Opentron Commands:")
        for cmd in self.protocol.commands():
            print(cmd)
    
    def clear_protocol(self):
        self.protocol._commands.clear()




# Initialize the OT-2
This step will take 1-2 minutes

In [4]:
protocol = execute.get_protocol_api('2.19') 
#protocol = simulate.get_protocol_api('2.19')  # Use simulate for testing
robot = OpentronMara(protocol)

# Execute pipetting command

This will command OT-2 to pipette exact volumes in the specified locations. After completion, the pipette tip will stay in its last position. Running this command again without resetting the commands using commands.clear() will instruct the robot to pick up a new pipette tip below the last used tip. 

In [13]:
robot.distribute_colors(df, ['volC', 'volM', 'volY', 'volK'])  

# If user interrupts the program, reset the robot

In [14]:
robot.pip300.drop_tip()  # Ensure the pipette is not holding a tip before starting

robot.protocol._commands.clear() # Clear the command queue
robot.tiprack_300.reset()  # Reset the tip racks to ensure a clean state

In [ ]:
robot.protocol.home()    # Home the robot if needed